In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/random-image-for-testing/image2.jpg
/kaggle/input/random-image-for-testing/image.jpg
/kaggle/input/pretrained-resnext/resnext.pth


In [4]:
import numpy, pandas, torch, torchvision, torchmetrics, sklearn
from platform import python_version

In [5]:
print(python_version())
print(numpy.__version__)
print(pandas.__version__)
print(torch.__version__)
print(torchvision.__version__)
print(torchmetrics.__version__)

3.7.12
1.21.6
1.3.5
1.11.0+cpu
0.12.0+cpu
0.11.0


In [6]:
try:
    import gradio as gr
except: 
    !pip -q install gradio
    import gradio as gr
    
print(f"Gradio version: {gr.__version__}")

Gradio version: 3.16.2


In [7]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



In [8]:
def create_resnext():
    model = models.resnext50_32x4d(pretrained=True)
  
    torch.manual_seed(42)
    torch.cuda.manual_seed(42)

    for param in model.parameters():
        param.requires_grad = False

    model.fc = nn.Sequential(
      nn.Linear(in_features=2048, out_features=80, bias=True)
  )

    return model

In [9]:
model = create_resnext()
model = nn.DataParallel(model)
model.load_state_dict(torch.load('/kaggle/input/pretrained-resnext/resnext.pth', map_location='cpu'))

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


  0%|          | 0.00/95.8M [00:00<?, ?B/s]

<All keys matched successfully>

In [10]:
classes = ['person',
  'bicycle',
  'car',
  'motorcycle',
  'airplane',
  'bus',
  'train',
  'truck',
  'boat',
  'traffic light',
  'fire hydrant',
  'stop sign',
  'parking meter',
  'bench',
  'bird',
  'cat',
  'dog',
  'horse',
  'sheep',
  'cow',
  'elephant',
  'bear',
  'zebra',
  'giraffe',
  'backpack',
  'umbrella',
  'handbag',
  'tie',
  'suitcase',
  'frisbee',
  'skis',
  'snowboard',
  'sports ball',
  'kite',
  'baseball bat',
  'baseball glove',
  'skateboard',
  'surfboard',
  'tennis racket',
  'bottle',
  'wine glass',
  'cup',
  'fork',
  'knife',
  'spoon',
  'bowl',
  'banana',
  'apple',
  'sandwich',
  'orange',
  'broccoli',
  'carrot',
  'hot dog',
  'pizza',
  'donut',
  'cake',
  'chair',
  'couch',
  'potted plant',
  'bed',
  'dining table',
  'toilet',
  'tv',
  'laptop',
  'mouse',
  'remote',
  'keyboard',
  'cell phone',
  'microwave',
  'oven',
  'toaster',
  'sink',
  'refrigerator',
  'book',
  'clock',
  'vase',
  'scissors',
  'teddy bear',
  'hair drier',
  'toothbrush']
        
print(len(classes))

80


In [11]:
from time import time

def predict(img):
    start = time()
    
    newimg = transform(img).unsqueeze(dim=0)
    model.eval()
    with torch.inference_mode():
        ylogit = model(newimg).detach()
        yprob = torch.sigmoid(ylogit)
    
    names_with_probs = {classes[i]: float(yprob[0][i]) for i in range(len(classes))}
    
    predtime = time() - start
    
    return names_with_probs, predtime

In [12]:
from PIL import Image
img = Image.open("/kaggle/input/random-image-for-testing/image.jpg")
a, b = predict(img)
print(a)
print(b)

{'person': 0.9961808919906616, 'bicycle': 0.008394231088459492, 'car': 0.0032457613851875067, 'motorcycle': 0.0007554020849056542, 'airplane': 2.6222582164336927e-05, 'bus': 0.0015153619460761547, 'train': 0.00045812051394023, 'truck': 0.0006146886735223234, 'boat': 0.003916422836482525, 'traffic light': 0.00107071443926543, 'fire hydrant': 4.716109742730623e-06, 'stop sign': 0.0003976893494836986, 'parking meter': 1.2496200449199487e-22, 'bench': 0.004423732403665781, 'bird': 0.0005676937871612608, 'cat': 0.00011253609409322962, 'dog': 0.0010629582684487104, 'horse': 7.284879757207818e-06, 'sheep': 9.622477591619827e-06, 'cow': 2.604585279186722e-05, 'elephant': 7.7267063858244e-09, 'bear': 7.681362035327766e-07, 'zebra': 3.7003861308448904e-08, 'giraffe': 4.587662090216327e-07, 'backpack': 0.042271628975868225, 'umbrella': 0.00140041159465909, 'handbag': 0.003939677029848099, 'tie': 0.0034668254666030407, 'suitcase': 4.9269237933913246e-05, 'frisbee': 0.0011718012392520905, 'skis': 6

In [21]:
import gradio as gr

example_list = [["/kaggle/input/random-image-for-testing/image.jpg"], ["/kaggle/input/random-image-for-testing/image2.jpg"]]
title = 'Multilabel Classifier'
description = "A ResNeXt-50 feature extractor computer vision model to identify objects present in an image out of 80 classes"
article = "Github repo-> https://github.com/Saatvik-Sinha/DSEG660-Multilabel-Classification-Challenge"

demo = gr.Interface(fn=predict,
                    inputs=gr.Image(type="pil"),
                    outputs=[gr.Label(num_top_classes=80, label="Predictions"),
                             gr.Number(label="Prediction time (s)")],
                    examples=example_list, 
                    title=title,
                    description=description,
                    article=article)

demo.launch(debug=False, share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://d6b7b25e-cca4-427f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


In [25]:
import os
import shutil

try:
    os.mkdir("/kaggle/working/Multilabel/")
except:
    shutil.rmtree("/kaggle/working/Multilabel/")
    os.mkdir("/kaggle/working/Multilabel/")

In [26]:
os.mkdir("/kaggle/working/Multilabel/examples")
for eg in example_list:
    name = eg[0].split('/')[-1]
    dst = "/kaggle/working/Multilabel/examples/" + name
    shutil.copy2(src=eg[0], dst=dst)

In [27]:
example_list = [["examples/" + example] for example in os.listdir("/kaggle/working/Multilabel/examples/")]
print(example_list)

[['examples/image2.jpg'], ['examples/image.jpg']]


In [30]:
shutil.copy2(src="/kaggle/input/pretrained-resnext/resnext.pth", 
                dst="/kaggle/working/Multilabel/resnext.pth")

'/kaggle/working/Multilabel/resnext.pth'

In [28]:
%%writefile "/kaggle/working/Multilabel/model.py"

import torch
from torchvision import models
from torch import nn

def create_resnext():
    model = models.resnext50_32x4d(pretrained=True)
  
    torch.manual_seed(42)
    torch.cuda.manual_seed(42)

    for param in model.parameters():
        param.requires_grad = False

    model.fc = nn.Sequential(
      nn.Linear(in_features=2048, out_features=80, bias=True)
  )

    return model

Writing /kaggle/working/Multilabel/model.py


In [31]:
%%writefile "/kaggle/working/Multilabel/app.py"
import os
import torch
from torch import nn
from model import create_resnext
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

model = create_resnext()
model = nn.DataParallel(model)
model.load_state_dict(torch.load('resnext.pth', map_location='cpu'))

classes = ['person',
  'bicycle',
  'car',
  'motorcycle',
  'airplane',
  'bus',
  'train',
  'truck',
  'boat',
  'traffic light',
  'fire hydrant',
  'stop sign',
  'parking meter',
  'bench',
  'bird',
  'cat',
  'dog',
  'horse',
  'sheep',
  'cow',
  'elephant',
  'bear',
  'zebra',
  'giraffe',
  'backpack',
  'umbrella',
  'handbag',
  'tie',
  'suitcase',
  'frisbee',
  'skis',
  'snowboard',
  'sports ball',
  'kite',
  'baseball bat',
  'baseball glove',
  'skateboard',
  'surfboard',
  'tennis racket',
  'bottle',
  'wine glass',
  'cup',
  'fork',
  'knife',
  'spoon',
  'bowl',
  'banana',
  'apple',
  'sandwich',
  'orange',
  'broccoli',
  'carrot',
  'hot dog',
  'pizza',
  'donut',
  'cake',
  'chair',
  'couch',
  'potted plant',
  'bed',
  'dining table',
  'toilet',
  'tv',
  'laptop',
  'mouse',
  'remote',
  'keyboard',
  'cell phone',
  'microwave',
  'oven',
  'toaster',
  'sink',
  'refrigerator',
  'book',
  'clock',
  'vase',
  'scissors',
  'teddy bear',
  'hair drier',
  'toothbrush']

from time import time

def predict(img):
    start = time()
    
    newimg = transform(img).unsqueeze(dim=0)
    model.eval()
    with torch.inference_mode():
        ylogit = model(newimg).detach()
        yprob = torch.sigmoid(ylogit)
    
    names_with_probs = {classes[i]: float(yprob[0][i]) for i in range(len(classes))}
    
    predtime = time() - start
    
    return names_with_probs, predtime

import gradio as gr

example_list = [["examples/" + example] for example in os.listdir("examples/")]

title = 'Multilabel Classifier'
description = "A ResNeXt-50 feature extractor computer vision model to identify objects present in an image out of 80 classes"
article = "Github repo-> https://github.com/Saatvik-Sinha/DSEG660-Multilabel-Classification-Challenge"

demo = gr.Interface(fn=predict,
                    inputs=gr.Image(type="pil"),
                    outputs=[gr.Label(num_top_classes=80, label="Predictions"),
                             gr.Number(label="Prediction time (s)")],
                    examples=example_list, 
                    title=title,
                    description=description,
                    article=article)

demo.launch()

Writing /kaggle/working/Multilabel/app.py


In [32]:
%%writefile "/kaggle/working/Multilabel/requirements.txt"

torch==1.11.0
torchvision==0.12.0
gradio==3.16.2

Writing /kaggle/working/Multilabel/requirements.txt


In [33]:
print(os.listdir("/kaggle/working/Multilabel"))

['examples', 'requirements.txt', 'resnext.pth', 'app.py', 'model.py']


In [34]:
shutil.make_archive('/kaggle/working/Multilabel',
                    'zip',
                    '/kaggle/working/',
                    'Multilabel')

'/kaggle/working/Multilabel.zip'